# WebParsing

В этом задании требуется обкачать интернет-магазин компьютерных игр ["GG.DEALS"](https://gg.deals/) с использованием библиотек [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) и/или [lxml](https://lxml.de/). Ваша программа должна скачать информацию о самых [рейтинговых](https://gg.deals/games/?sort=metascore&type=1) 300 играх магазина. 



## Общий подход к решению задачи

Задачу условно можно разделить на два этапа. На первом этапе требуется получить ссылки на все нужные игры из раздела, на втором – получить информацию о каждой из игр.

### Этап 1. Получение ссылок на игры

Для получения всех ссылок на игры нужно обкачать первые страницы [раздела](https://gg.deals/games/?sort=metascore&type=1). Переключаясь между страницами раздела можно заметить, как меняется URL страницы (появляется параметр `page`). Варьируя это значение в диапазоне можно получить ссылки на все требуемые страницы.

### Этап 2. Получение информации об игре

Требуется извлечь следующие элементы:

0. поле "url" – url страницы из адресной строки браузера;
1. поле "name" – название игры;
2. поле "image" – ссылка на постер игры; поле "market_url" – ссылка на игру в оригинальном магазине (см. View On Steam, например);
3. поля "wishlist_count", "alert_count", "owners_count" – значения соответвующих счетчиков.
4. группы полей, если имеются:
	* "release_date" – дата релиза (выхода) игры;
	* "developer" – разработчик игры;
	* "metacritic_score" – рейтинг Metascore;
	* "user_score" – рейтинг Userscore;
	* "review_label", "review_positive_pctg", "review_count" – общий пользовательский вердикт (например, Very Positive), доля позитивных обзоров, общее число обзоров на игру;
	* "genres" – список жанров игры;
	* "tags" – список тегов игры;
	* "features" – список особенностей игры.
5. поле "dlcs" – список ссылок на DLC (дополнения) к игре, поле "packs" – список ссылок на Packs (расширенные версии игр); списки могут быть пустыми;
6. поле "pc_systems" – список поддерживаемых ОС компьютеров;
7. поле "price_history" – список цен на игру в оригинальных магазинах (голубая линия) за весь имеющийся период. 
 
Список цен на игру должен представлять собой питоновский список словарей, каждый словарь должен иметь три поля:
* "ts" – время изменения цены;
* "price" – новая цена (в рублях);
* "shop" – имя магазина.

In [1]:
import json
import requests
from lxml import etree, html as lhtml
from bs4 import BeautifulSoup
import ast

from multiprocessing.dummy import Pool, Queue
from tqdm import tqdm
import gzip
import codecs

In [3]:
firstPart = 'https://gg.deals'

In [4]:
def genre_make(dc):
    genre = dc.xpath('//a[contains(@href, "/games/?genre=")]/text()')
    if (genre):
        return [i for i in genre if(i.isalpha())]
    else:
        return None

In [5]:
def dict_make(ind):
    
    if (ind):
        headers_dict = {"x-requested-with": "XMLHttpRequest"}
        res = requests.get("https://gg.deals/ru/games/chartHistoricalData/{}/?hideKeyshops=0".format(ind),
                           headers = headers_dict)
        part = res.text.split('"deals":')
        if (part and len(part) >=2 ):
            part2 = part[1].split('"offers":')
            part3 = part2[0][:-1].replace('"x"','"ts"').replace('"y"','"price"')
            if (part3 != '[]'):
                ans = json.loads(part3)
                for i in ans:
                    i.pop('name')
                return ans
            else:
                return None
        else:
            return None
    else:
        return None

In [6]:
def image_make(dc):
    res = dc.xpath('//img[contains(@class, "image-game")]/@srcset')
    if (res):
        image = res[0]
        image = image.split(',')
        return [i.split(' ')[0] for i in image]
    else:
        return None

In [7]:
def none_check(parent):
    if parent:
        return True
    return None

In [8]:
def dlcs_packs(ind, type_d):
    headers_dict = {"x-requested-with": "XMLHttpRequest"}
    page = requests.get('https://gg.deals/ru/games/relations/{}/?type={}&offset=0&hideKeyshops=0'.format(ind, type_d),
                            headers=headers_dict)
    soup = BeautifulSoup(page.text, 'html.parser')
    res = soup.find_all('a', class_ = "game-info-title")
    if (res):
        return [firstPart + i.attrs['href'] for i in res]
    else:
        return None

In [9]:
def process_page(url):
    try:
        r = get_page(url)

        dc = lhtml.fromstring(r.text)
        soup = BeautifulSoup(r.text, 'html.parser')

        name = dc.xpath('//span[@itemprop="name"]/text()')
        image = image_make(dc)
        market_url = soup.find('a', class_ = 'score-grade')
        wishlist = soup.find_all("span", class_="count")
        dlms = soup.find('section', id = 'game-dlcs')
        packs = soup.find('section', id = 'game-packs')
        release_date = dc.xpath('//p[contains(@class, "game-info-details-content")]/text()')
        metacritic = soup.find('a', class_ = 'score-circle score-metascore')
        user = soup.find('a', class_ = 'score-circle score-userscore')
        review = soup.find('span', class_ = 'reviews-label')
        tags = dc.xpath('//a[contains(@href, "/games/?tag=")]/text()') 
        features = dc.xpath('//a[contains(@href, "/games/?feature=")]/text()')
        check = soup.find('div', class_ = 'game-info-actions')
        p = soup.find('div', class_ = "game-requirements-tabs")
        if (none_check(check)):
            a = check.find('div',
                            class_ = lambda s: s and s.startswith('game-collection-actions')).attrs['data-counters-url']
            ind = a[:-1].split('/')[-1]
        else:
            ind = None

        if (p):
            pc = [i.text for i in p.find_all('li', class_ = lambda s: s and s.endswith('menu-item'))] 
        else:
            pc = None

        game_info = {
            'url': url, 
            'name': name[-1] if name else None,
            'image': image[0] if (image) else None,
            "market_url":  none_check(market_url) and market_url.attrs['href'],
            "wishlist_count": none_check(wishlist) and int(wishlist[0].text),
            "alert_count": none_check(wishlist) and int(wishlist[-3].text),
            "owners_count": none_check(wishlist) and int(wishlist[-1].text),
            "release_date": none_check(release_date) and release_date[0],
            "developer": none_check(release_date) and release_date[1],
            "metacritic_score": none_check(metacritic) and float(metacritic.find('span', class_ = 'overlay').text),
            "user_score": none_check(user) and float(user.find('span', class_ = 'overlay').text),
            "review_label": none_check(review) and
                                        (review.text).split(' (', maxsplit = 1)[0],
            "review_positive_pctg": none_check(review) and
                                       int(review.attrs['title'].split('%', maxsplit = 1)[0]),
            "review_count": none_check(review) and
                                       int((review.text).split(' (', maxsplit = 1)[1][:-1].replace(',','')),
            "genres": genre_make(dc),
            "tags": tags if tags else None,
            "features": features if features else None,
            "dlcs": dlcs_packs(ind,'dlc'),
            "packs": dlcs_packs(ind, 'packs'),
            "pc_systems": pc,
            "price_history": dict_make(ind)
        }
        
        filtered = dict(filter(lambda item: item[1] is not None, game_info.items()))
        return filtered
    except Exception as ex:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
    return None

In [10]:
def get_page(url, attempts = 5):
    for i in range(attempts):
        response = requests.get(url)
        if response.status_code == 200:
            return response
        else:
            print(response.status_code)
        sleep(5)
    print("Sorry, url was nt donwloaded: {}".format(url))

In [11]:
def add_to_queue(page_ind, game_ind):
    page = get_page('https://gg.deals/games/?sort=metascore&type=1&page={}'.format(page_ind + 1))
    
    if page:
        doc = lhtml.fromstring(page.text)
        for game in doc.xpath('//a[contains(@class, "game-link")]/attribute::href', limit = game_ind):
            queue.put(firstPart + game)

In [12]:
def create_queue(num_pages = 300, games = 24):
    block_numb = num_pages // games + 1
    last_block = block_numb * games - num_pages

    arg = zip(range(block_numb), [games] * (block_numb - 1) + [last_block])
    with Pool(processes=10) as pool:
        urls = pool.starmap(add_to_queue, arg)
    pool.join()

In [13]:
queue = Queue()
create_queue()
print(queue.qsize())

# doc = lhtml.fromstring(ret.text)

def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb+') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            record = process_page(queue.get())
            if (record == None):
                break
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)

with Pool(processes=10) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))
    

  0%|                                                                                          | 0/312 [00:00<?, ?it/s]

312


100%|████████████████████████████████████████████████████████████████████████████████| 312/312 [02:48<00:00,  1.85it/s]
